In [ ]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [ ]:
import os, sys
sys.path.append(os.path.join(os.path.dirname(os.path.abspath(os.getcwd())), "lib/python"))

import numpy as np

from picopic.plot_builder import PlotBuilder
from picopic.h5_reader import H5Reader


In [ ]:
##  configuration options
data_path = '/home/cosmonaut/dev/pic/picopic'

timestamp=1e-11
show_grid=False
use_cache=False
cmap='terrain'
image_interpolation = 'nearest'
autoselect = True

cmp_0 = 'E/r'
cmp_1 = 'E/z'
cmp_beam = 'density'
specie_beam = 'beam_electrons'

if os.path.isfile(data_path) or os.path.isfile(os.path.join(data_path, "data.h5")):
    reader = H5Reader(path = data_path, use_cache=use_cache, verbose=True)
else:
    raise EnvironmentError("There is no corresponding data/metadata files in the path " + data_path + ". Can not continue.")

# update shape to get custom shaped images
# useful to get just part of frame
# or get frames, which has smaller shape than full frame
size=[0, 0, reader.meta.geometry_grid[0], reader.meta.geometry_grid[1]]

r_scale = (size[2] - size[0]) / reader.meta.geometry_grid[0]
z_scale = (size[3] - size[1]) / reader.meta.geometry_grid[1]

# color limits (WARNING: clim_estimation may works incorrectly)
clim_estimation = reader.meta.get_clim_estimation()
clim_0 = [-clim_estimation, clim_estimation]
clim_1 = [-clim_estimation, clim_estimation]
clim_beam = [0, reader.meta.beams[0].bunch_density]

x_axis_label = r'$\mathit{Z (m)}$'
y_axis_label = r'$\mathit{R (m)}$'
cbar_axis_label = r'$\frac{V}{m}$'
cbar_bunch_density_axis_label = r'$m^{-3}$'

plot_name_0 = r'$\mathbf{Electric\enspace Field\enspace Radial \enspace Component\enspace (E_r)}$'
plot_name_1 = r'$\mathbf{Electric\enspace Field\enspace Longitudial \enspace Component\enspace (E_z)}$'
plot_name_beam = r'$\mathbf{Electrons\enspace Beam\enspace Density \enspace  (\rho_{beam})}$'

In [ ]:
# get data
data_0 = []
data_1 = []
data_beam = []
shape = 'rec'
for probe in reader.meta.probes:
    frame = reader.meta.get_frame_number_by_timestamp(timestamp, probe.schedule)
    if (probe.shape == shape) and (probe.size[0] == size[0]) and (probe.size[1] == size[1]) and(probe.size[2] == size[2]) and(probe.size[3] == size[3]):
        if probe.component == cmp_0: data_0 = reader.rec(cmp_0, size, frame)
        if probe.component == cmp_1: data_1 = reader.rec(cmp_1, size, frame)
        if probe.component == cmp_beam and probe.specie == specie_beam: data_beam = reader.rec("{}/{}".format(cmp_beam, specie_beam), size, frame)
        timestamp = frame * probe.schedule * reader.meta.time[2]

# try bigger frames, if autoselect enabled
if (len(data_0) == 0 or len(data_1) == 0 or len(data_beam) == 0) and autoselect:
    for probe in reader.meta.probes:
        frame = reader.meta.get_frame_number_by_timestamp(timestamp, probe.schedule)
        if (probe.shape == shape) and (probe.size[0] <= size[0]) and (probe.size[1] <= size[1]) and(probe.size[2] >= size[2]) and(probe.size[3] >= size[3]):
            if probe.component == cmp_0 and len(data_0) == 0: data_0 = reader.rec("{}".format(cmp_0), probe.size, frame)[size[0]:size[2], size[1]:size[3]]
            if probe.component == cmp_1 and len(data_1) == 0: data_1 = reader.rec("{}".format(cmp_1), probe.size, frame)[size[0]:size[2], size[1]:size[3]]
            if probe.component == cmp_beam and probe.specie == specie_beam and len(data_beam) == 0: data_beam = reader.rec("{}/{}".format(cmp_beam, specie_beam), probe.size, frame)[size[0]:size[2], size[1]:size[3]]
            timestamp = frame * probe.schedule * reader.meta.time[2]

In [ ]:
# define plot builder
plot = PlotBuilder(size[3] - size[1], size[2] - size[0],
                   fig_color=reader.meta.figure_color, 
                   fig_width=reader.meta.figure_width,
                   fig_height=reader.meta.figure_height, 
                   fig_dpi=reader.meta.figure_dpi,
                   font_family=reader.meta.figure_font_family,
                   font_name=reader.meta.figure_font_name,
                   font_size=reader.meta.figure_font_size,
                   
                   x_ticklabel_end=reader.meta.geometry_size[1] * z_scale,
                   y_ticklabel_end=reader.meta.geometry_size[0] * r_scale,
                   tickbox=True, grid=show_grid, is_invert_y_axe=False,
                   aspect='equal', image_interpolation=image_interpolation, 
                   guess_number_ticks = 20)

In [ ]:
# add subplots
plot.add_subplot_cartesian_2d(plot_name_0, 311, x_axe_label=x_axis_label, y_axe_label=y_axis_label)
plot.add_subplot_cartesian_2d(plot_name_1, 312, x_axe_label=x_axis_label, y_axe_label=y_axis_label)
plot.add_subplot_cartesian_2d(plot_name_beam, 313, x_axe_label=x_axis_label, y_axe_label=y_axis_label)

# add timestamp to plot
plot.get_figure().suptitle("Time: {:.2e} s".format(timestamp), x=.85, y=.95)

# add images
plot.add_image(plot_name_0, data_0, cmap=cmap, clim=clim_0)
plot.add_image(plot_name_1, data_1, cmap=cmap, clim=clim_1)
plot.add_image(plot_name_beam, data_beam, cmap="{}_r".format(cmap), clim=clim_beam)

# add colorbars
plot.add_colorbar(plot_name_0, ticks=clim_0, title=cbar_axis_label)
plot.add_colorbar(plot_name_1, ticks=clim_1, title=cbar_axis_label)
plot.add_colorbar(plot_name_beam, ticks=clim_beam, title=cbar_bunch_density_axis_label)

plot.show()